In [1]:
import pandas as pd
from google.transit import gtfs_realtime_pb2
import requests
import zipfile
from datetime import datetime as dt
from datetime import timedelta
from numpy import nan, isnan
import duckdb
import io
import os
from pathlib import Path

In [2]:
data_path = "exploration_duckdb.db"

In [3]:
my_file = Path(data_path)

if my_file.is_file():
    os.remove(data_path)

duckdb.connect(data_path)
print("-------  DUCK DATABASE CREATED --------")

-------  DUCK DATABASE CREATED --------


In [4]:
con = duckdb.connect(data_path)

In [5]:
url = "https://ara-api.enroute.mobi/rla/gtfs/vehicle-positions"
response = requests.get(url)

feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

records = []
for entity in feed.entity:
    if not entity.HasField("vehicle"):
        continue

    vehicle = entity.vehicle
    trip = vehicle.trip
    pos = vehicle.position

    record = {
        "trip_id": trip.trip_id if trip.HasField("trip_id") else None,
        "route_id": trip.route_id if trip.HasField("route_id") else None,
        "latitude": pos.latitude if pos.HasField("latitude") else None,
        "longitude": pos.longitude if pos.HasField("longitude") else None,
        "bearing": pos.bearing if pos.HasField("bearing") else None,
        "timestamp": vehicle.timestamp if vehicle.HasField("timestamp") else None,
        "stop_id": vehicle.stop_id if vehicle.HasField("stop_id") else None,
        "vehicle_id": vehicle.vehicle.id if vehicle.HasField("vehicle") else None,
    }
    records.append(record)

df_vehicle_position = pd.DataFrame(records)

con.sql("CREATE TABLE vehicle_positions AS SELECT * FROM df_vehicle_position")

con.sql("INSERT INTO vehicle_positions SELECT * FROM df_vehicle_position")

print("--------------- RETURN SELECT ------------------------")
con.sql("SELECT * FROM vehicle_positions").show()

--------------- RETURN SELECT ------------------------
┌──────────────────────────────────────────────────────────────┬──────────┬────────────────────┬────────────────────┬─────────┬────────────┬─────────┬────────────┐
│                           trip_id                            │ route_id │      latitude      │     longitude      │ bearing │ timestamp  │ stop_id │ vehicle_id │
│                           varchar                            │ varchar  │       double       │       double       │ double  │   int64    │ varchar │  varchar   │
├──────────────────────────────────────────────────────────────┼──────────┼────────────────────┼────────────────────┼─────────┼────────────┼─────────┼────────────┤
│ 5838889-22_R_95_2212_16:30-PROJET2025-22-Semaine-20          │ 22       │  43.76307678222656 │  7.198024749755859 │   209.0 │ 1758638524 │ 2994    │ PDV110806  │
│ 6366084-20_A_50_2003_16:12-SETP2025-20-Semaine-42            │ 20       │ 43.686790466308594 │ 7.1837263107299805 │   150.0

In [6]:

url = "https://ara-api.enroute.mobi/rla/gtfs/trip-updates"
response = requests.get(url)

feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

trips_update_list = []

for entity in feed.entity:
    if entity.HasField("trip_update"):
        trip = entity.trip_update.trip

        trip_id = trip.trip_id
        route_id = trip.route_id
        direction_id = trip.direction_id if trip.HasField("direction_id") else None

        for stop_time_update in entity.trip_update.stop_time_update:
            stop_id = stop_time_update.stop_id
            stop_sequence = stop_time_update.stop_sequence

            if stop_time_update.HasField(
                "arrival"
            ) and stop_time_update.arrival.HasField("time"):
                arrival_time = stop_time_update.arrival.time
            else:
                arrival_time = None

            if stop_time_update.HasField(
                "departure"
            ) and stop_time_update.departure.HasField("time"):
                departure_time = stop_time_update.departure.time
            else:
                departure_time = None

            trips_update_list.append(
                {
                    "trip_id": trip_id,
                    "route_id": route_id,
                    "direction_id": direction_id,
                    "stop_id": stop_id,
                    "stop_sequence": stop_sequence,
                    "arrival_time": dt.fromtimestamp(arrival_time).time() if arrival_time else None,
                    "arrival_date": dt.fromtimestamp(arrival_time).date() if arrival_time else None,
                    "departure_time": dt.fromtimestamp(departure_time).time() if departure_time else None,
                    "departure_date":  dt.fromtimestamp(departure_time).date() if departure_time else None,
                }
            )


df_trips_update = pd.DataFrame(trips_update_list)

con.sql("CREATE TABLE trip_update AS SELECT * FROM df_trips_update")

con.sql("INSERT INTO trip_update SELECT * FROM df_trips_update")

print("--------------- RETURN SELECT ------------------------")
con.sql("SELECT * FROM trip_update").show()


--------------- RETURN SELECT ------------------------
┌─────────────────────────────────────────────────────┬──────────┬──────────────┬─────────┬───────────────┬──────────────┬──────────────┬────────────────┬────────────────┐
│                       trip_id                       │ route_id │ direction_id │ stop_id │ stop_sequence │ arrival_time │ arrival_date │ departure_time │ departure_date │
│                       varchar                       │ varchar  │    double    │ varchar │     int64     │     time     │     date     │      time      │      date      │
├─────────────────────────────────────────────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼──────────────┼────────────────┼────────────────┤
│ 6416041-21_R_99_2105_18:36-SETP2025-21-Semaine-40   │ 21       │          1.0 │ 4283    │             0 │ NULL         │ NULL         │ 18:36:00       │ 2025-09-23     │
│ 6416041-21_R_99_2105_18:36-SETP2025-21-Semaine-40   │ 21       │          1.0 │ 320

In [7]:

url = "https://chouette.enroute.mobi/api/v1/datas/OpendataRLA/gtfs.zip"
response = requests.get(url)
response.raise_for_status()

    
files_not_wanted = ["agency.txt","feed_info.txt"]
dfs_static = {}

with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    for filename in z.namelist():
        if filename not in files_not_wanted:

            with z.open(filename) as f:
                df = pd.read_csv(f)

            table_name = filename.replace(".txt","")

            

            print(f"---------- {table_name} -------------")

            match table_name :

                case "calendar":
                
                    df['start_date'] = df['start_date'].apply(lambda x :  dt.strptime(str(x), "%Y%m%d").date() if not isnan(x) else nan)
                    df['end_date'] = df['end_date'].apply(lambda x :  dt.strptime(str(x), "%Y%m%d").date() if not isnan(x) else nan)

                case "stop_times":

                    df['arrival_time'] = df['arrival_time'].apply(lambda x :"00" + x[2:] if not pd.isnull(x) and x[:2] == "24" else x)
                    df['arrival_time'] = df['arrival_time'].apply(lambda x : dt.strptime(x, "%H:%M:%S").time() if not pd.isnull(x) else nan)

                    df['departure_time'] = df['departure_time'].apply(lambda x :"00" + x[2:] if not pd.isnull(x) and x[:2] == "24" else x)
                    df['departure_time'] = df['departure_time'].apply(lambda x : dt.strptime(x, "%H:%M:%S").time() if not  pd.isnull(x) else nan)

                case "calendar_dates":
                
                    df['date'] = df['date'].apply(lambda x :  dt.strptime(str(x), "%Y%m%d").date() if not isnan(x) else nan)


            dfs_static[table_name] = df

            con.sql(f"CREATE TABLE {table_name} AS SELECT * FROM df")

            con.sql(f"INSERT INTO {table_name} SELECT * FROM df")
            
            # con.sql(f"SELECT * FROM {table_name}").show()


---------- stops -------------
---------- routes -------------
---------- trips -------------
---------- stop_times -------------
---------- calendar -------------
---------- calendar_dates -------------
---------- shapes -------------


In [8]:
# df_trips_update
# df_vehicle_position
# dfs_static['calendar']
# dfs_static['stops']
# dfs_static['stop_times']
# dfs_static['calendar_dates']
# dfs_static['routes']
# dfs_static['shapes']
# dfs_static['trips']

In [9]:
dans = dt.now() - timedelta(hours=2)
print(dans)

2025-09-23 14:43:02.321249


In [10]:
sql_request = """
SHOW TABLES
"""
con.sql(sql_request).show()

┌───────────────────┐
│       name        │
│      varchar      │
├───────────────────┤
│ calendar          │
│ calendar_dates    │
│ routes            │
│ shapes            │
│ stop_times        │
│ stops             │
│ trip_update       │
│ trips             │
│ vehicle_positions │
└───────────────────┘



In [11]:
sql_request = """
SELECT * FROM vehicle_positions
"""

con.sql(sql_request).show()

┌──────────────────────────────────────────────────────────────┬──────────┬────────────────────┬────────────────────┬─────────┬────────────┬─────────┬────────────┐
│                           trip_id                            │ route_id │      latitude      │     longitude      │ bearing │ timestamp  │ stop_id │ vehicle_id │
│                           varchar                            │ varchar  │       double       │       double       │ double  │   int64    │ varchar │  varchar   │
├──────────────────────────────────────────────────────────────┼──────────┼────────────────────┼────────────────────┼─────────┼────────────┼─────────┼────────────┤
│ 5838889-22_R_95_2212_16:30-PROJET2025-22-Semaine-20          │ 22       │  43.76307678222656 │  7.198024749755859 │   209.0 │ 1758638524 │ 2994    │ PDV110806  │
│ 6366084-20_A_50_2003_16:12-SETP2025-20-Semaine-42            │ 20       │ 43.686790466308594 │ 7.1837263107299805 │   150.0 │ 1758638524 │ 5082    │ RLA334     │
│ 6420102-85_A_4

In [12]:
sql_request = """
SELECT * FROM trip_update
"""

con.sql(sql_request).show()

┌─────────────────────────────────────────────────────┬──────────┬──────────────┬─────────┬───────────────┬──────────────┬──────────────┬────────────────┬────────────────┐
│                       trip_id                       │ route_id │ direction_id │ stop_id │ stop_sequence │ arrival_time │ arrival_date │ departure_time │ departure_date │
│                       varchar                       │ varchar  │    double    │ varchar │     int64     │     time     │     date     │      time      │      date      │
├─────────────────────────────────────────────────────┼──────────┼──────────────┼─────────┼───────────────┼──────────────┼──────────────┼────────────────┼────────────────┤
│ 6416041-21_R_99_2105_18:36-SETP2025-21-Semaine-40   │ 21       │          1.0 │ 4283    │             0 │ NULL         │ NULL         │ 18:36:00       │ 2025-09-23     │
│ 6416041-21_R_99_2105_18:36-SETP2025-21-Semaine-40   │ 21       │          1.0 │ 32057   │             1 │ 18:36:56     │ 2025-09-23   │ 18

In [13]:
sql_request = """
WITH retard AS(
SELECT trip_update.trip_id, trip_update.stop_id, trip_update.arrival_time,stop_times.arrival_time 
AS theoric_arrival_time, date_diff('minutes',trip_update.arrival_time,theoric_arrival_time) AS retard
FROM trip_update 
INNER JOIN stop_times 
ON trip_update.trip_id = stop_times.trip_id
)
SELECT trip_id, avg(retard)
FROM retard
GROUP BY trip_id
"""

con.sql(sql_request).show()
retard_df =  con.sql(sql_request).df()

┌─────────────────────────────────────────────────────────────┬─────────────────────┐
│                           trip_id                           │     avg(retard)     │
│                           varchar                           │       double        │
├─────────────────────────────────────────────────────────────┼─────────────────────┤
│ 6402555-32_A_48_3201_16:47-SETP2025-32-Semaine-34           │                 1.7 │
│ 6416444-33_A_50_3303_16:49-SETP2025-33-L-Ma-J-V-36          │ -0.4023809523809524 │
│ 6261666-73_R_95_7302_19:41-PROJET2025-73-Semaine-16         │                NULL │
│ 6368790-09_R_98_0907_16:40-PROJET2025-09-Semaine-35         │   8.026785714285714 │
│ 6368792-09_R_98_0901_17:42-PROJET2025-09-Semaine-35         │ -1.3984962406015038 │
│ 6416354-51_A_49_5101_17:58-PROJET2025-51-Semaine-10         │   1.244047619047619 │
│ 6422653-99_A_50_9901_18:15-SETP2025-99-Semaine-11           │  1.8571428571428572 │
│ 6422654-99_A_50_9902_16:54-SETP2025-99-Semaine-11   

In [15]:
sql_request = """
WITH retard AS(
SELECT trip_update.trip_id, date_diff('minutes',trip_update.arrival_time,stop_times.arrival_time) AS retard
FROM trip_update 
INNER JOIN stop_times 
ON trip_update.trip_id = stop_times.trip_id
)
SELECT vehicle_positions.trip_id, vehicle_id,latitude,longitude,avg(retard.retard)
FROM vehicle_positions
INNER JOIN retard
ON vehicle_positions.trip_id = retard.trip_id
GROUP BY vehicle_positions.trip_id, vehicle_id,latitude,longitude

"""

con.sql(sql_request).show()

┌──────────────────────────────────────────────────────────────┬────────────┬────────────────────┬────────────────────┬─────────────────────┐
│                           trip_id                            │ vehicle_id │      latitude      │     longitude      │ avg(retard.retard)  │
│                           varchar                            │  varchar   │       double       │       double       │       double        │
├──────────────────────────────────────────────────────────────┼────────────┼────────────────────┼────────────────────┼─────────────────────┤
│ 6412132-37_A_50_3702_16:33-SETP2025-37-Semaine-37            │ RLA20      │ 43.710697174072266 │   7.26356315612793 │  5.9393939393939394 │
│ 6416068-21_A_50_2106_16:04-SETP2025-21-Semaine-40            │ RLA311     │ 43.664955139160156 │  7.154240608215332 │ -22.152298850574713 │
│ 6369768-42_A_6_4201_16:24-PROJET2025-42-Semaine-83           │ TCA108190  │ 43.664031982421875 │  7.151033401489258 │               -9.75 │
│ 6420

In [16]:
sql_request = """
WITH retard AS (
SELECT trip_update.stop_id, date_diff('minutes',trip_update.arrival_time,stop_times.arrival_time) AS retard
FROM trip_update 
INNER JOIN stop_times 
ON trip_update.trip_id = stop_times.trip_id
)
SELECT stops.stop_id,stops.stop_lat,stops.stop_lon,avg(retard.retard)
FROM stops 
INNER JOIN retard
ON retard.stop_id = stops.stop_id
GROUP BY stops.stop_id,stops.stop_lat,stops.stop_lon
"""

con.sql(sql_request).show()

┌─────────┬───────────┬──────────┬─────────────────────┐
│ stop_id │ stop_lat  │ stop_lon │ avg(retard.retard)  │
│ varchar │  double   │  double  │       double        │
├─────────┼───────────┼──────────┼─────────────────────┤
│ 220     │ 43.700654 │ 7.251172 │  -7.899441340782123 │
│ 2529    │ 43.722792 │ 7.290672 │                18.0 │
│ 421     │ 43.714739 │  7.28165 │                NULL │
│ 1009    │ 43.727078 │ 7.285195 │ -0.3753581661891118 │
│ 1155    │ 43.708367 │ 7.283413 │   7.181818181818182 │
│ 775     │ 43.725504 │ 7.288101 │ 0.29042904290429045 │
│ 3186    │ 43.722671 │ 7.107156 │ -11.165413533834586 │
│ 21654   │ 43.756667 │ 7.171343 │              3.8875 │
│ 21655   │ 43.757075 │ 7.174957 │              3.8875 │
│ 21223   │  43.75958 │ 7.175855 │   4.531914893617022 │
│  ·      │      ·    │     ·    │           ·         │
│  ·      │      ·    │     ·    │           ·         │
│  ·      │      ·    │     ·    │           ·         │
│ 5170    │ 43.695606 │ 7.17482

In [17]:
sql_request = """
WITH retard AS(
SELECT trip_update.trip_id, trip_update.route_id, trip_update.stop_id, trip_update.arrival_time,stop_times.arrival_time 
AS theoric_arrival_time, date_diff('minutes',trip_update.arrival_time,theoric_arrival_time) AS retard
FROM trip_update 
INNER JOIN stop_times 
ON trip_update.trip_id = stop_times.trip_id
)
SELECT route_id, avg(retard) as moyenne_retards
FROM retard
GROUP BY route_id
ORDER BY moyenne_retards
"""

con.sql(sql_request).show()

┌────────────────────────────────────────────────────────┬─────────────────────┐
│                        route_id                        │   moyenne_retards   │
│                        varchar                         │       double        │
├────────────────────────────────────────────────────────┼─────────────────────┤
│ 90                                                     │  -45.73504273504273 │
│ 91                                                     │  -19.16944024205749 │
│ 83                                                     │ -10.399124726477025 │
│ CN                                                     │  -9.833333333333334 │
│ 98                                                     │  -8.158088235294118 │
│ CE                                                     │  -4.857142857142857 │
│ chouette:Line:07759d26-564a-4043-bd2d-d1e0c98d686c:LOC │ -3.0458815028901736 │
│ 51                                                     │ -2.2953688970772967 │
│ B                         

In [18]:
sql_request = """
WITH retard AS(
SELECT trip_update.trip_id, date_diff('minutes',trip_update.arrival_time,stop_times.arrival_time) AS retard
FROM trip_update 
INNER JOIN stop_times 
ON trip_update.trip_id = stop_times.trip_id
)
SELECT vehicle_positions.trip_id, vehicle_id,retard.retard
FROM vehicle_positions
INNER JOIN retard
ON vehicle_positions.trip_id = retard.trip_id

"""

con.sql(sql_request).show()

┌─────────────────────────────────────────────────────┬────────────┬────────┐
│                       trip_id                       │ vehicle_id │ retard │
│                       varchar                       │  varchar   │ int64  │
├─────────────────────────────────────────────────────┼────────────┼────────┤
│ 4350591-40_A_50_4001_16:32-RESEAU2023-40-Semaine-04 │ TCA117299  │    -17 │
│ 4350591-40_A_50_4001_16:32-RESEAU2023-40-Semaine-04 │ TCA117299  │    -15 │
│ 4350591-40_A_50_4001_16:32-RESEAU2023-40-Semaine-04 │ TCA117299  │    -14 │
│ 4350591-40_A_50_4001_16:32-RESEAU2023-40-Semaine-04 │ TCA117299  │    -13 │
│ 4350591-40_A_50_4001_16:32-RESEAU2023-40-Semaine-04 │ TCA117299  │    -11 │
│ 4350591-40_A_50_4001_16:32-RESEAU2023-40-Semaine-04 │ TCA117299  │     -8 │
│ 4350591-40_A_50_4001_16:32-RESEAU2023-40-Semaine-04 │ TCA117299  │     -6 │
│ 5097933-69_A_46_6901_16:40-PROJET2023-69-Semaine-15 │ PDV76      │    -17 │
│ 5097933-69_A_46_6901_16:40-PROJET2023-69-Semaine-15 │ PDV76   